In [8]:
%pip install -q tensorflow matplotlib

ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import PIL.Image
import requests
from io import BytesIO


ModuleNotFoundError: No module named 'tensorflow'

In [7]:
# --- FUNCTION DEFINITIONS ---

def load_image(url, max_dim=512):
    response = requests.get(url)
    img = PIL.Image.open(BytesIO(response.content))
    img = img.convert('RGB')
    long = max(img.size)
    scale = max_dim / long
    img = img.resize((round(img.size[0] * scale), round(img.size[1] * scale)), PIL.Image.ANTIALIAS)
    img = np.array(img)
    img = img.astype(np.float32)[np.newaxis, ...] / 255.
    return img

def tensor_to_image(tensor):
    tensor = tensor*255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor)>3:
        tensor = tensor[0]
    return PIL.Image.fromarray(tensor)

In [ ]:
# --- LOAD IMAGES ---
content_url = 'https://images.unsplash.com/photo-1503023345310-bd7c1de61c7d'  # city landscape
style_url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg'

content_image = load_image(content_url)
style_image = load_image(style_url)

plt.subplot(1, 2, 1)
plt.title('Content Image')
plt.imshow(content_image[0])
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title('Style Image')
plt.imshow(style_image[0])
plt.axis('off')
plt.show()

In [ ]:
from keras import backend as K

target_image = K.constant(preprocess_image(target_image_path))
style_reference_image = K.constant(preprocess_image(style_reference_image_path))

# This placeholder will contain our generated image
combination_image = K.placeholder((1, img_height, img_width, 3))

# We combine the 3 images into a single batch
input_tensor = K.concatenate([target_image,
                              style_reference_image,
                              combination_image], axis=0)

# We build the VGG19 network with our batch of 3 images as input.
# The model will be loaded with pre-trained ImageNet weights.
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet',
                    include_top=False)
print('Model loaded.')

In [ ]:
# --- LOAD PRETRAINED MODEL ---
import tensorflow_hub as hub
hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

# --- APPLY STYLE TRANSFER ---
stylized_image = hub_model(tf.constant(content_image), tf.constant(style_image))[0]

# --- DISPLAY RESULT ---
output_image = tensor_to_image(stylized_image)
plt.imshow(output_image)
plt.title("Stylized Output")
plt.axis('off')
plt.show()
